In [5]:
import requests as rq
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
from fake_useragent import UserAgent


URL = 'https://journal.tinkoff.ru/flows/business-all'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         ' AppleWebKit/537.36 (KHTML, like Gecko)'
                         ' Chrome/100.0.4896.88 Safari/537.36'}

HOST = 'https://journal.tinkoff.ru/'
FILE = '/tinkoffjournal.csv'






In [9]:
def get_html(url: str) -> bytes:
    return rq.get(url, headers={'User-Agent': UserAgent().chrome}, timeout=15).content

In [137]:
get_flows_args = {'flow_inner_args': {'name': 'div', 'attrs': {'class': 'inner--K9Svv'}},
                  'flow_topic_args': {'name':'a', 'attrs': {'class': 'item--ALUvj'}},
                  'flow_title_args': {'name': 'h4','attrs': {'class': 'heading--lf0qy'}}
                  }

def get_flows(html: bytes, args: dict) -> pd.DataFrame:
    '''
        Парсинг страницы с потоками https://journal.tinkoff.ru/flows/
        Собирает: названия потоков, топики потоков, ссылки на страницы топиков
    '''
    soup = bs(html,'html.parser')
    print((args['flow_inner_args']))
    items = soup.find_all(**(args['flow_inner_args']))[:2]
    print(f'Всего flows: {len(items)}')
    flow_dict = {'flow_title': [], 'start_topic': [], 'flow_link': []}
    for item in tqdm(items):

        flow_items = item.find_all(**(args['flow_topic_args']))[:2]
        
        for flow_item in flow_items:
            flow_dict['start_topic'].append(flow_item.text)
            flow_dict['flow_link'].append(f"https://journal.tinkoff.ru{flow_item.get('href')}posts/")

        flow_dict['flow_title'] += [item.find(**(args['flow_title_args'])).text] * len(flow_items)
    return pd.DataFrame(flow_dict)


        
flows_page_link = "https://journal.tinkoff.ru/flows/"

html = get_html(flows_page_link)

#не везде нужно добавлять posts !!!!
flow_df = get_flows(html, get_flows_args)
flow_df

{'name': 'div', 'attrs': {'class': 'inner--K9Svv'}}
Всего flows: 2


100%|██████████| 2/2 [00:00<00:00, 3453.52it/s]


,flow_title,start_topic,flow_link
0,Инвестиции,Инвестиции,https://journal.tinkoff.ru/flows/invest/posts/
1,Инвестиции,Инвестидеи,https://journal.tinkoff.ru/flows/invest-ideas/...
2,Путешествия,Путешествия,https://journal.tinkoff.ru/flows/travel/posts/
3,Путешествия,Путешествия по миру,https://journal.tinkoff.ru/flows/world-travel/...


In [139]:
get_flow_news_args = {'article_card_args': {'name': 'div', 'attrs': {'class': 'item--LA1zO'}},
                      'article_link_args': {'name': 'a', 'attrs': {'class': 'link--aKZVS'}},
                      'article_author_args': {'name': 'div', 'attrs': {'class': 'name--ur745'}},
                      'article_date_args': {'name': 'time', 'attrs': {'class':'date--ZZJXU'}},
                      'article_views_args': {'name': 'span', 'attrs': {'class': 'counter--sXVCe'}},
                      'article_title_args': {'name': 'h3', 'attrs': {'class': 'title--Oe3sZ'}},
                      'article_likes_args': {'name': 'span', 'attrs': {'class': 'counter--fwxAj'}},
                      'article_comments_count_args': {'name': 'span', 'attrs': {'class': 'content--WdSlu'}},
                      'article_saves_count_args': {'name': 'button', 'attrs': {'class': 'favorites--y85P0'}},
                      'topic_args': {'name': 'h1', 'attrs': {'class': 'heading--lf0qy'}}
                      }

def get_flow_news(html: bytes, args: dict, links_visited: set) -> pd.DataFrame:
  '''
    Парсинг страницы потока
  '''
  soup = bs(html,'html.parser')
  items = soup.find_all(**(args['article_card_args']))[:3]
  news_dict = {'author': [], 'date': [], 'views': [],
                  'topic': [], 'title': [], 'link': [],
                'likes':[], 'comments_count':[], 'saves_count': []}
    
  print(f'Новостей на странице: {len(items)}')
  bad_items = []
  skipped_news_count = 0
  for item in tqdm(items):
    
    try:
      article_link = 'https://journal.tinkoff.ru' + item.find(**(args['article_link_args'])).get('href')
      if article_link not in links_visited:
        links_visited.add(article_link)
        news_dict['author'].append(item.find(**(args['article_author_args'])).text)
        news_dict['date'].append(item.find(**(args['article_date_args'])).get('datetime'))
        news_dict['views'].append(item.find(**(args['article_views_args'])).text)
        news_dict['title'].append(item.find(**(args['article_title_args'])).text)
        news_dict['link'].append(article_link)
        news_dict['likes'].append(item.find(**(args['article_likes_args'])).text)
        news_dict['comments_count'].append(item.find(**(args['article_comments_count_args'])).text)
        news_dict['saves_count'].append(item.find(**(args['article_saves_count_args'])).text)
      else:
        skipped_news_count += 1
        continue

    except Exception:
        skipped_news_count += 1
        bad_items.append(item)
        continue
  news_dict['topic'] = [soup.find(**(args['topic_args'])).text] * (len(items) - skipped_news_count)
  return pd.DataFrame(news_dict), bad_items

page_link = 'https://journal.tinkoff.ru/flows/invest/posts/page/1'

#html = get_html(page_link)
#df, bad_items = get_flow_news(html, get_flow_news_args)
#df.head()

In [151]:
def parse_flow(flow_link: str, args, links_visited: set):
    flow_df = pd.DataFrame({'author': [], 'date': [], 'views': [],
                            'topic': [], 'title': [], 'link': [],
                            'likes':[], 'comments_count':[], 'saves_count': [], 'flow_page_link': [], 'flow_link': []})
    for page in tqdm(range(1, 2)):
        print('Парсинг страничек')
        time.sleep(3)
        try:
            page_link = f'{flow_link}page/{page}'
            html = get_html(page_link)
            flow_page_df, bad_items = get_flow_news(html, args, links_visited)
            flow_page_df['flow_page_link'] = [page_link]*flow_page_df.shape[0]
            flow_page_df['flow_link'] = [flow_link]*flow_page_df.shape[0]
            flow_df = pd.concat([flow_df, flow_page_df], ignore_index=True)
            
        except Exception:
            continue

    return flow_df

def pase_flows(flows_links: np.ndarray, args):
    '''
        Парсинг страниц потоков по их ссылкам
    '''
    flows_df = pd.DataFrame({'author': [], 'date': [], 'views': [],
                            'topic': [], 'title': [], 'link': [],
                            'likes':[], 'comments_count':[], 'saves_count': [], 'flow_page_link': []})
    links_visited = set()
    for flow_link in tqdm(flows_links):
        print(len(links_visited))
        print('парсинг потока')
        try:   

            flow_df = parse_flow(flow_link, args, links_visited)
            flows_df = pd.concat([flows_df, flow_df], ignore_index=True)
            
        except Exception:
            continue
    return flows_df

#flow_df = parse_flow()
    

In [105]:
flow_df['flow_href'].values

array(['https://journal.tinkoff.ru/flows/invest/posts/',
       'https://journal.tinkoff.ru/flows/invest-ideas/posts/',
       'https://journal.tinkoff.ru/flows/travel/posts/',
       'https://journal.tinkoff.ru/flows/world-travel/posts/'],
      dtype=object)

In [152]:
d = pase_flows(flow_df['flow_link'].values, get_flow_news_args)

  0%|          | 0/4 [00:00<?, ?it/s]

0
парсинг потока


Парсинг страничек
Новостей на странице: 3



 25%|██▌       | 1/4 [00:03<00:10,  3.42s/it]

3
парсинг потока


...

Парсинг страничек
Новостей на странице: 3



 75%|███████▌  | 3/4 [00:10<00:03,  3.49s/it]

5
парсинг потока


Парсинг страничек
Новостей на странице: 3



100%|██████████| 4/4 [00:14<00:00,  3.62s/it]


In [124]:
d

,author,date,views,topic,title,link,likes,comments_count,saves_count,flow_page_link,flow_link
0,Юлия Федорович,2024-03-07,1K,Инвестиции,Как я инвестирую через краудлендинговые площад...,https://journal.tinkoff.ru/a-kak-invest-kraudl...,3,9,1,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/...
1,Алексей Климовский,2024-03-07,7K,Инвестиции,"Как я вложил более 1,2 млн рублей и стабильно ...",https://journal.tinkoff.ru/invested-more-than-...,5,16,3,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/...
2,Алексей Климовский,2024-03-05,70K,Инвестиции,"Как я положил 800 000 ₽ на вклад под 18,5% год...",https://journal.tinkoff.ru/a-kak-invest-800k-n...,26,77,12,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/...
3,Ольга,2024-03-07,4K,Путешествия,Как мы съездили в Тулу из Москвы на один день,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/,10,6,3,https://journal.tinkoff.ru/flows/travel/posts/...,https://journal.tinkoff.ru/flows/travel/posts/...
4,Сергей Червоненко,2024-03-07,787,Путешествия,Как я бесплатно переночевал в стамбульском She...,https://journal.tinkoff.ru/besplatno-perenoche...,4,4,2,https://journal.tinkoff.ru/flows/travel/posts/...,https://journal.tinkoff.ru/flows/travel/posts/...


In [153]:
def get_article_content(link: np.ndarray):
    html = get_html(link)
    soup = bs(html,'html.parser')
    article_text = soup.find('div', attrs = {'class':'_articleView_1v9h1_1'}).text
    article_author = soup.find(lambda tag: (tag.name == 'a' or tag.name == 'div')\
                               and tag.get('class') == ['_author_1qoqa_6']).text
    topic_name = soup.find('a', attrs = {'class':'_flow_1xwjy_45'}).text
    return [article_text.replace(article_author, ''), topic_name]

def get_articles_content(article_links: list[str]) -> pd.DataFrame:
    articles_contens = [] 
    for article_link in article_links:
        t = get_article_content(article_link)
        t.append(article_link)
        articles_contens.append(t)
    return pd.DataFrame(articles_contens, columns=['article_text', 'topic_name', 'link'])


In [154]:
articles_content_df = get_articles_content(d['link'].values)
articles_content_df.head()

,article_text,topic_name,link
0,"Я коммерческий автор, и в середине 2023 года н...",Инвестиции,https://journal.tinkoff.ru/a-kak-invest-kraudl...
1,📈 Мои актуальные результаты на двух крупнейших...,А как инвестировать,https://journal.tinkoff.ru/invested-more-than-...
2,Недавно у меня освободились средства с предыду...,Инвестиции,https://journal.tinkoff.ru/a-kak-invest-800k-n...
3,"Всем привет, хочу рассказать, чем можно себя з...",Читатели путешествуют,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/
4,Возникла у меня необходимость отправиться в ст...,Читатели путешествуют,https://journal.tinkoff.ru/besplatno-perenoche...


In [113]:
d

,author,date,views,topic,title,link,likes,comments_count,saves_count,flow_page_link
0,Юлия Федорович,2024-03-07,1K,Инвестиции,Как я инвестирую через краудлендинговые площад...,https://journal.tinkoff.ru/a-kak-invest-kraudl...,3,9,1,https://journal.tinkoff.ru/flows/invest/posts/...
1,Алексей Климовский,2024-03-07,8K,Инвестиции,"Как я вложил более 1,2 млн рублей и стабильно ...",https://journal.tinkoff.ru/invested-more-than-...,5,17,3,https://journal.tinkoff.ru/flows/invest/posts/...
2,Алексей Климовский,2024-03-05,70K,Инвестиции,"Как я положил 800 000 ₽ на вклад под 18,5% год...",https://journal.tinkoff.ru/a-kak-invest-800k-n...,26,79,12,https://journal.tinkoff.ru/flows/invest/posts/...
3,Ольга,2024-03-07,4K,Путешествия,Как мы съездили в Тулу из Москвы на один день,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/,10,6,3,https://journal.tinkoff.ru/flows/travel/posts/...
4,Сергей Червоненко,2024-03-07,787,Путешествия,Как я бесплатно переночевал в стамбульском She...,https://journal.tinkoff.ru/besplatno-perenoche...,4,4,2,https://journal.tinkoff.ru/flows/travel/posts/...


In [114]:
articles_content_df

,article_text,topic_name,link
0,"Я коммерческий автор, и в середине 2023 года н...",Инвестиции,https://journal.tinkoff.ru/a-kak-invest-kraudl...
1,📈 Мои актуальные результаты на двух крупнейших...,А как инвестировать,https://journal.tinkoff.ru/invested-more-than-...
2,Недавно у меня освободились средства с предыду...,Инвестиции,https://journal.tinkoff.ru/a-kak-invest-800k-n...
3,"Всем привет, хочу рассказать, чем можно себя з...",Читатели путешествуют,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/
4,Возникла у меня необходимость отправиться в ст...,Читатели путешествуют,https://journal.tinkoff.ru/besplatno-perenoche...


In [155]:
merged_df = d.merge(articles_content_df, on='link', how='left')
merged_df


,author,date,views,topic,title,link,likes,comments_count,saves_count,flow_page_link,flow_link,article_text,topic_name
0,Юлия Федорович,2024-03-07,1K,Инвестиции,Как я инвестирую через краудлендинговые площад...,https://journal.tinkoff.ru/a-kak-invest-kraudl...,3,9,1,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/,"Я коммерческий автор, и в середине 2023 года н...",Инвестиции
1,Алексей Климовский,2024-03-07,8K,Инвестиции,"Как я вложил более 1,2 млн рублей и стабильно ...",https://journal.tinkoff.ru/invested-more-than-...,5,17,3,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/,📈 Мои актуальные результаты на двух крупнейших...,А как инвестировать
2,Алексей Климовский,2024-03-05,70K,Инвестиции,"Как я положил 800 000 ₽ на вклад под 18,5% год...",https://journal.tinkoff.ru/a-kak-invest-800k-n...,26,79,12,https://journal.tinkoff.ru/flows/invest/posts/...,https://journal.tinkoff.ru/flows/invest/posts/,Недавно у меня освободились средства с предыду...,Инвестиции
3,Ольга,2024-03-07,4K,Путешествия,Как мы съездили в Тулу из Москвы на один день,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/,10,6,3,https://journal.tinkoff.ru/flows/travel/posts/...,https://journal.tinkoff.ru/flows/travel/posts/,"Всем привет, хочу рассказать, чем можно себя з...",Читатели путешествуют
4,Сергей Червоненко,2024-03-07,787,Путешествия,Как я бесплатно переночевал в стамбульском She...,https://journal.tinkoff.ru/besplatno-perenoche...,4,4,2,https://journal.tinkoff.ru/flows/travel/posts/...,https://journal.tinkoff.ru/flows/travel/posts/,Возникла у меня необходимость отправиться в ст...,Читатели путешествуют


In [156]:
merged_df['flow_page_link'].values

array(['https://journal.tinkoff.ru/flows/invest/posts/page/1',
       'https://journal.tinkoff.ru/flows/invest/posts/page/1',
       'https://journal.tinkoff.ru/flows/invest/posts/page/1',
       'https://journal.tinkoff.ru/flows/travel/posts/page/1',
       'https://journal.tinkoff.ru/flows/travel/posts/page/1'],
      dtype=object)

In [157]:
merged_df['flow_link'].values

array(['https://journal.tinkoff.ru/flows/invest/posts/',
       'https://journal.tinkoff.ru/flows/invest/posts/',
       'https://journal.tinkoff.ru/flows/invest/posts/',
       'https://journal.tinkoff.ru/flows/travel/posts/',
       'https://journal.tinkoff.ru/flows/travel/posts/'], dtype=object)

In [158]:
flow_df['flow_link'].values

array(['https://journal.tinkoff.ru/flows/invest/posts/',
       'https://journal.tinkoff.ru/flows/invest-ideas/posts/',
       'https://journal.tinkoff.ru/flows/travel/posts/',
       'https://journal.tinkoff.ru/flows/world-travel/posts/'],
      dtype=object)

In [159]:
flow_df.merge(merged_df, on='flow_link', how='left')

,flow_title,start_topic,flow_link,author,date,views,topic,title,link,likes,comments_count,saves_count,flow_page_link,article_text,topic_name
0,Инвестиции,Инвестиции,https://journal.tinkoff.ru/flows/invest/posts/,Юлия Федорович,2024-03-07,1K,Инвестиции,Как я инвестирую через краудлендинговые площад...,https://journal.tinkoff.ru/a-kak-invest-kraudl...,3,9,1,https://journal.tinkoff.ru/flows/invest/posts/...,"Я коммерческий автор, и в середине 2023 года н...",Инвестиции
1,Инвестиции,Инвестиции,https://journal.tinkoff.ru/flows/invest/posts/,Алексей Климовский,2024-03-07,8K,Инвестиции,"Как я вложил более 1,2 млн рублей и стабильно ...",https://journal.tinkoff.ru/invested-more-than-...,5,17,3,https://journal.tinkoff.ru/flows/invest/posts/...,📈 Мои актуальные результаты на двух крупнейших...,А как инвестировать
2,Инвестиции,Инвестиции,https://journal.tinkoff.ru/flows/invest/posts/,Алексей Климовский,2024-03-05,70K,Инвестиции,"Как я положил 800 000 ₽ на вклад под 18,5% год...",https://journal.tinkoff.ru/a-kak-invest-800k-n...,26,79,12,https://journal.tinkoff.ru/flows/invest/posts/...,Недавно у меня освободились средства с предыду...,Инвестиции
3,Инвестиции,Инвестидеи,https://journal.tinkoff.ru/flows/invest-ideas/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Путешествия,Путешествия,https://journal.tinkoff.ru/flows/travel/posts/,Ольга,2024-03-07,4K,Путешествия,Как мы съездили в Тулу из Москвы на один день,https://journal.tinkoff.ru/v-tulu-iz-moskvy-trip/,10,6,3,https://journal.tinkoff.ru/flows/travel/posts/...,"Всем привет, хочу рассказать, чем можно себя з...",Читатели путешествуют
5,Путешествия,Путешествия,https://journal.tinkoff.ru/flows/travel/posts/,Сергей Червоненко,2024-03-07,787,Путешествия,Как я бесплатно переночевал в стамбульском She...,https://journal.tinkoff.ru/besplatno-perenoche...,4,4,2,https://journal.tinkoff.ru/flows/travel/posts/...,Возникла у меня необходимость отправиться в ст...,Читатели путешествуют
6,Путешествия,Путешествия по миру,https://journal.tinkoff.ru/flows/world-travel/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:


# Парсер актуальных новостей
def get_content(html: bytes) -> pd.DataFrame:
    soup = bs(html,'html.parser')
    items = soup.find_all('div', attrs = {'class':'item--b9CGu'})
    news_dict = {'author': [], 'date': [], 'views': [],
                  'topic': [], 'title': [], 'link': [],
                'likes':[], 'comments_count':[], 'saves_amount': []}
    
    print(f'Новостей на странице: {len(items)}')
    bad_items = []

    for item in tqdm(items[:4]):
        time.sleep(3)
        try:
            news_dict['author'].append(item.find('div', attrs = {'class':'name--H8jp2'}).text)
            news_dict['date'].append(item.find('time', attrs = {'class':'date--BiQBs'}).get('datetime'))
            news_dict['views'].append(item.find('div', attrs = {'class':'views--AwSbN'}).text)
            news_dict['topic'].append(item.find('a', attrs = {'class':'flow--Sf0iX'}).text)
            news_dict['title'].append(item.find('h3', attrs = {'class':'title--h5Y_9'}).text)
            news_dict['link'].append(item.find('a', attrs = {'class':'link--hGana'}).get('href'))
            news_dict['likes'].append(item.find('span', attrs = {'class':'counter--JSDtN'}).text)
            news_dict['comments_count'].append(item.find('span', attrs = {'class':'content--corUP'}).text)
            news_dict['saves_amount'].append(item.find('button', attrs = {'class':'favorites--BW3AB'}).text)
        except Exception:
            bad_items.append(item)
            continue
        
    return pd.DataFrame(news_dict), bad_items

page_link = "https://journal.tinkoff.ru/tag/breaking-news/page/1/"

html = get_html(page_link)
df, bad_items = get_content(html)
df.head()